# Training Script

In [1]:
import os, object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2023-03-09 12:33:57.069906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Set up file structure and download chosen base model from Model Zoo

In [2]:
modelName = "640x640_model"
pretrainedModel = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
pretrainedModelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"

# Script sourced from Tensorflow: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/tensorflow-1.14/training.html
tfRecordScript = "generate_tfrecord.py"
labelMap = "label_map.pbtxt"

In [3]:
filePaths = {
    "WORKSPACE": os.path.join("TF", "workspace"),
    "SCRIPTS": os.path.join("TF", "scripts"),
    "APIMODEL": os.path.join("TF", "models"),
    "ANNOTATIONS": os.path.join("TF", "workspace", "annotations"),
    "IMAGES": os.path.join("TF", "workspace", "images"),
    "PRETRAINED_MODELS": os.path.join("TF", "workspace", "pretrained_models"),
}

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', modelName, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(filePaths['SCRIPTS'], tfRecordScript),
    'LABELMAP': os.path.join(filePaths['ANNOTATIONS'], labelMap)
}

In [5]:
for path in filePaths.values():
    !mkdir -p {path}

In [6]:
if not os.path.exists(os.path.join(filePaths['APIMODEL'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {filePaths['APIMODEL']}

Cloning into 'TF/models'...
remote: Enumerating objects: 81991, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 81991 (delta 293), reused 400 (delta 254), pack-reused 81533
Receiving objects: 100% (81991/81991), 596.38 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (58482/58482), done.


In [8]:
!wget {pretrainedModelURL}
!mv {pretrainedModel+'.tar.gz'} {filePaths['PRETRAINED_MODELS']}
!cd {filePaths['PRETRAINED_MODELS']} && tar -zxvf {pretrainedModel+'.tar.gz'}

--2023-03-09 12:40:10--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.200.16, 2a00:1450:4009:822::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.200.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  17.7MB/s    in 1.1s    

2023-03-09 12:40:11 (17.7 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x640_coco17_tp

# Create label mapping

In [ ]:
objectLabels = [{'name':'resistor', 'id':1},
                {'name':'electrolytic_capacitor', 'id':2},
                {'name':'LED', 'id':3},
                {'name':'ceramic_capacitor', 'id':4},
                {'name':'IC', 'id':5}]

